# Orbit Determination

## Import Module

In [ ]:
import importlib, sys, subprocess

packages = "matplotlib, numpy, sympy, pandas, astropy".split(', ')
for package in packages :
    if not importlib.util.find_spec(package):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])

from math import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sy

from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u
from datetime import datetime

## Define Vector
벡터 a, b와 실수 k에 대해,

- 정의: a = Vector(a.x, a.y, a.z)

- 크기: a.norm

- 단위벡터: a.hat

- 성분: a.x, a.y, a.z

- 실수배: a * k, a / k

- 덧셈, 뺄셈, 역벡터: a + b, a - b, -a

- 내적: a @ b

- 외적: a * b

In [37]:
class Vector:
    _isrealnumber = lambda x : isinstance(x, int) or isinstance(x, float)
    __slots__ = ('x', 'y', 'z')
    def __init__(self, x, y, z = 0.0):
        if all(map(Vector._isrealnumber, (x, y, z))):
            self.x, self.y, self.z = map(float, (x, y, z))
    def __repr__(self):
        return f'Vector({self.x:.04f}, {self.y:.04f}, {self.z:.04f})'
    def norm(self):
        return (self.x**2 + self.y**2 + self.z**2)**0.5
    @property
    def norm(self):
        return (self.x**2 + self.y**2 + self.z**2)**0.5
    @norm.setter
    def norm(self, value):
        self.x, self.y, self.z = value, 0, 0
    @property
    def _comps(self):
        return (self.x, self.y, self.z)
    @property
    def norm(self):
        return sum(i*i for i in self._comps) ** 0.5
    def __repr__(self):
        return "Vector({:.04f}, {:.04f}, {:.04f})".format(*self._comps)
    def multiplied(self, k):
        return Vector(*(i*k for i in self._comps))
    def __mul__(self, k):
        if Vector._isrealnumber(k):
            return self.multiplied(k)
    def __rmul__(self, k):
        return self*k
    def __truediv__(self, k):
        if Vector._isrealnumber(k):
            return self.multiplied(1/k)
    def __neg__(self):
        return Vector(*(-i for i in self._comps))
    def __add__(self, other):
        return Vector(*(i+j for i, j in zip(self._comps, other._comps)))
    def __radd__(self, other):
        return self._add(other) 
    def __sub__(self, other):
        return self + -other
    def __rsub__(self, other):
        return -self + other
    def dot(self, other):
        return sum(i*j for i,j in zip(self._comps, other._comps))
    def cross(self, other):
        return Vector(self.y * other.z - self.z * other.y, self.z * other.x - self.x * other.z, self.x * other.y - self.y * other.x)
    def __mul__(self, other):
        if Vector._isrealnumber(other):
            return self.multiplied(other)
        if isinstance(other, Vector):
            return self.cross(other)
    def __matmul__(self, other):
        if isinstance(other, Vector):
            return self.dot(other)
    @property
    def hat(self):
        return self * (1/self.norm)

## Input

In [38]:
Data = [['2023-09-27 13:47:12','02:50:40.40','14:55:07.5'],       # Example Source) Jupiter in Stellarium
        ['2023-10-02 13:47:12','02:49:04.10','14:47:20.9'], 
        ['2023-10-06 13:47:12','02:47:35.43','14:40:17.9']]     # UTC obs time, RA, Dec

## Set System

In [39]:
lam = 127.00543     # 관측자 경도[deg]
phi = 37.30881      # 관측자 위도[deg]
H = 0.098           # 관측자 고도[km]

mu = 1.32712440018e+11   # 중력변수[km^3 s^-2]
eps = 23 + 26/60                        # 황도경사[deg]

R_eq = 6378         # 지구 적도반지름[km]
f = 1/299           # 지구타원체 편평도

km_to_AU = 1/1.496e+8   # km to AU

I_hat = Vector(1,0,0)   # 황도좌표계 기본벡터
J_hat = Vector(0,1,0)
K_hat = Vector(0,0,1)

i_hat = I_hat           # 적도좌표계 기본벡터
j_hat = Vector(0,cos(radians(eps)),-sin(radians(eps))) 
k_hat = Vector(0,sin(radians(eps)),cos(radians(eps)))

## Data Organization

In [46]:
t, RaDec, theta = [], [], []
for data in Data:
  t.append(datetime.strptime(data[0],'%Y-%m-%d %H:%M:%S'))
  
  RAh,RAm,RAs = map(float,data[1].split(':'))
  Decd,Decm,Decs = map(float,data[2].split(':'))
  if Decd<0:  Decm,Decs = -Decm,-Decs
  RaDec.append([(RAh + RAm/60 + RAs/3600)*360/24, Decd + Decm/60 + Decs/3600])

  obs_time = Time(data[0], scale = 'utc', location = EarthLocation(lat = phi*u.deg, lon = lam*u.deg))
  Lst = float(str(obs_time.sidereal_time('mean')*u.deg).replace(' deg hourangle',''))*360/24
  theta.append(Lst)

## Earth Location

In [41]:
R_E_vecs = []

def nrange(x):
    while x<0:  x += 2*pi
    while x>=2*pi:  x -= 2*pi
    return x

Q0 = [1.00000011,0.01671022,0.00005,-11.26064,102.94719,100.46435]
Qdot = [-0.00000005,-0.00003804,-46.94/3600,-18228.25/3600,1198.28/3600,129597740.63/3600]

for data in Data:
    JD = Time(str(data[0])[:-3], format = 'iso', out_subfmt = 'date_hm').jd
    T0 = (JD - 2451545) / 36525

    [a_E,e_E,i_E,Om_E,vom_E,L_E] = [Q0[i] + Qdot[i]*T0 for i in range(6)]
    a_E /= km_to_AU
    om_E,M_E = nrange(radians(vom_E - Om_E)), nrange(radians(L_E - vom_E))
    i_E,Om_E = radians(i_E),nrange(radians(Om_E))

    E_E = M_E + e_E/2
    if M_E > pi:
        E_E = M_E - e_E/2
    while True:
        ratio = (E_E - e_E*sin(E_E) - M_E) / (1 - e_E * cos(E_E))
        if abs(ratio) <= 0.00001:
            break
        E_E = E_E - ratio
    theta_E = nrange(2*atan(sqrt((1-e_E) / (1+e_E)) * tan(E_E/2)))

    QxX = [[cos(Om_E)*cos(om_E) - sin(Om_E)*sin(om_E)*cos(i_E), -cos(Om_E)*sin(om_E) - sin(Om_E)*cos(i_E)*cos(om_E), sin(Om_E)*sin(i_E)],
           [sin(Om_E)*cos(om_E) + cos(Om_E)*cos(i_E)*sin(om_E), -sin(Om_E)*sin(om_E) + cos(Om_E)*cos(i_E)*cos(om_E), -cos(Om_E)*sin(i_E)],
           [sin(i_E)*sin(om_E), sin(i_E)*cos(om_E), cos(i_E)]]
    R_E_vec = a_E*(1 - e_E**2) / (1 + e_E*cos(theta_E)) * Vector(QxX[0][0]*cos(theta_E) + QxX[0][1]*sin(theta_E), QxX[1][0]*cos(theta_E) + QxX[1][1]*sin(theta_E), QxX[2][0]*cos(theta_E) + QxX[2][1]*sin(theta_E))
    R_E_vecs.append(R_E_vec)

## Observer Location

In [42]:
R_phi = R_eq / sqrt(1 - (2*f - f**2) * sin(radians(phi))**2)
R_e_vecs = [(R_phi + H)*cos(radians(phi))*(cos(radians(theta))*i_hat + sin(radians(theta))*j_hat) + (R_phi*(1 - f)**2 + H)*sin(radians(phi))*k_hat for theta in theta]

R_vecs = [R_E_vecs[i] + R_e_vecs[i] for i in [0,1,2]]

## Gauss Method

In [43]:
rho_hats = [(cos(radians(radec[1]))*(cos(radians(radec[0]))*i_hat + sin(radians(radec[0]))*j_hat) + sin(radians(radec[1]))*k_hat).hat for radec in RaDec]

tau0,tau2,tau = [float((t[i] - t[j]).days)*86400 + float((t[i] - t[j]).seconds) for [i,j] in [[0,1],[2,1],[2,0]]]
p_vecs = [rho_hats[i] * rho_hats[j] for [i,j] in [[1,2],[0,2],[0,1]]]

D0 = rho_hats[0] @ p_vecs[0]
D = [[R_vec @ p_vec for p_vec in p_vecs] for R_vec in R_vecs]


A = 1/D0 * (-D[0][1]*tau2/tau + D[1][1] + D[2][1]*tau0/tau)
B = 1/(6*D0) * (D[0][1]*(tau2**2 - tau**2)*tau2/tau + D[2][1]*(tau**2 - tau0**2)*tau0/tau)
E = R_vecs[1] @ rho_hats[1]

a = -(A**2 + 2*A*E + R_vecs[1].norm**2)
b = -2*mu*B*(A+E)
c = -(mu*B)**2

x = sy.symbols('x',positive = True)
r1 = float(sy.solve(x**8 + a*x**6 + b*x**3 + c, x)[-1])

rhos = [float(1/D0 * ((6*(D[2][0]*tau0/tau2 + D[1][0]*tau/tau2)*r1**3 + mu*D[2][0]*(tau**2 - tau0**2)*tau0/tau2) / (6*r1**3 + mu*(tau**2 - tau2**2)) - D[0][0])),
       float(A + mu*B/r1**3),
       float(1/D0 * ((6*(D[0][2]*tau2/tau0 - D[1][2]*tau/tau0)*r1**3 + mu*D[0][2]*(tau**2 - tau2**2)*tau2/tau0) / (6*r1**3 + mu*(tau**2 - tau2**2)) - D[2][2]))]

r_vecs = [R_vecs[i] + rhos[i]*rho_hats[i] for i in [0,1,2]]

f0 = 1 - (mu * tau0**2) / (2 * r1**3)
f2 = 1 - (mu * tau2**2) / (2 * r1**3)
g0 = tau0 - (mu * tau0**3) / (6 * r1**3)
g2 = tau2 - (mu * tau2**3) / (6 * r1**3)

v1_vec = (-f2*r_vecs[0] + f0*r_vecs[2]) / (f0*g2 - f2*g0)

r_vec = r_vecs[1]
v_vec = v1_vec

## Orbit Elements

In [44]:
r = r_vec.norm
v = v_vec.norm
v_r = v_vec @ r_vec.hat

k_vec = r_vec * v_vec; k = k_vec.norm
N_vec = K_hat * k_vec; N = N_vec.norm

e_vec = -(k_vec * v_vec/mu + r_vec.hat); e = e_vec.norm

a = (k**2/mu) / (1 - e**2)

i = acos(k_vec.z/k)*180/pi
if i > 180:
    i = 360 - i

Om = acos(N_vec.x/N)*180/pi
if N_vec.y < 0:
    Om = 360 - Om

om = acos((N_vec @ e_vec)/(N * e))*180/pi
if e_vec.z < 0:
    om = 360 - om

theta = acos((r_vec @ e_vec)/(r * e))*180/pi
if v_r < 0:
    theta = 360 - theta

print(f'\
이심률: {e}\n\
장반경: {a * km_to_AU} AU\n\
경사각: {i} deg\n\
승교점 경도: {Om} deg\n\
근심점 이각: {om} deg\n\
진근점 이각: {theta} deg\n')      # 궤도요소가 이상하게 나오는 문제가 발생합니다.

이심률: 0.8111083209273133
장반경: 13.608723923997925 AU
경사각: 1.383837377776339 deg
승교점 경도: 95.02818087243737 deg
근심점 이각: 38.87170335888013 deg
진근점 이각: 264.1285295932779 deg

